# GLOBAL SHARK ATTACK - Database Cleaning and Visualization
#### SOURCE : https://www.kaggle.com/teajay/global-shark-attacks

## Objective

This notebook aims to demonstrate a way to clean and visuzalize the distribution and patterns of the most relevant columns from the database provided



## Perguntas

## Premissas

A register to be valid needs to be identifiable.

## Conclusão

In [2]:
import pandas as pd
import re
import os

In [3]:
df = pd.read_csv('attacks.csv', encoding = 'latin-1')

### First things first. Let´s take a look at what we have here:

In [4]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

### In order to facilitate manipulation and avoid errors: 

In [5]:
df.columns = [str(column)
               .lower()
               .replace("(Y/N)", "")
               .replace("Investigator or ", "")
               .strip()
               .replace(" ", "_")          
           for column in df.columns]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_number             8702 non-null   object 
 1   date                    6302 non-null   object 
 2   year                    6300 non-null   float64
 3   type                    6298 non-null   object 
 4   country                 6252 non-null   object 
 5   area                    5847 non-null   object 
 6   location                5762 non-null   object 
 7   activity                5758 non-null   object 
 8   name                    6092 non-null   object 
 9   sex                     5737 non-null   object 
 10  age                     3471 non-null   object 
 11  injury                  6274 non-null   object 
 12  fatal_(y/n)             5763 non-null   object 
 13  time                    2948 non-null   object 
 14  species                 3464 non-null 

### From the cell above is possible to notice: 

* Most of the columns contain __'object'__ data type, even for columns  like 'age' and 'date'. Which raises the hypothesis that the registers do not follow a pattern

* The majority of registers are null. *(__25723__ entries x __8702__ 'case_number' non null registers)* 

* 'case_number' has considerably bigger number of non-null recording than the second larger column. __(8702 x 6309)__

#### TO DO: 

* Exclude register that are entirely null  
* Explore more the column 'case_number':

In [7]:
df.dropna(how='all', inplace= True)

In [8]:
df['case_number'].nunique()

6287

In [9]:
df['case_number'].unique()

array(['2018.06.25', '2018.06.18', '2018.06.09', ..., 'ND.0001', '0',
       'xx'], dtype=object)

In [10]:
ID = df.groupby('case_number')
ID['case_number'].count().sort_values(ascending=False)

case_number
0               2400
1980.07.00         2
2006.09.02         2
2005.04.06         2
2009.12.18         2
                ... 
1959.07.30         1
1959.07.28         1
1959.07.25.b       1
1959.07.25.a       1
xx                 1
Name: case_number, Length: 6287, dtype: int64

### We see:

* There are several items on 'case_number' column that are duplicated

* Lots of registers with "0".

* Different patterns of input. Some of them are probably related to the date.

### Exploring 'case_number' == 0 :

In [11]:
df[df['case_number']==0]

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23


In [12]:
# value 0 is a string
df[df['case_number']=='0'].head()

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
6302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6304.0,NaN,NaN
6303,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6305.0,NaN,NaN
6304,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6306.0,NaN,NaN
6305,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6307.0,NaN,NaN
6306,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6308.0,NaN,NaN


In [13]:
# checking out other columns
df[df['case_number']=='0'].count()

case_number               2400
date                         0
year                         0
type                         0
country                      0
area                         0
location                     0
activity                     0
name                         0
sex                          0
age                          0
injury                       0
fatal_(y/n)                  0
time                         0
species                      0
investigator_or_source       0
pdf                          0
href_formula                 0
href                         0
case_number.1                0
case_number.2                0
original_order               7
unnamed:_22                  0
unnamed:_23                  0
dtype: int64

### As we can see, for case_number == 0 :

* There are not enough field filled to identify a occurence. So those registers will be deleted

In [14]:
invalid = df['case_number'] == "0"
df.drop(df.loc[invalid].index, inplace = True)

### Then we have:

In [15]:
df.shape

(6303, 24)

In [16]:
unicos = df['case_number'].nunique()
unicos

6286

In [17]:
duplicados = df[df.duplicated(subset='case_number')]['case_number'].count()
duplicados

16

In [18]:
unicos + duplicados -  len(df)

-1

### There is probably a null value for 'case_number'
___unique values + duplicated - total = -1___

In [19]:
df[df['case_number'].isnull()]

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
5488,NaN,Reported 06-Sep-1905,1905.0,Provoked,USA,Florida,"Fort Pierce, St Lucie County",Fishing,David Curry,M,...,NaN,"Muncie Evening Press, 8/6/1913",1905.09.06.R-Curry.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1905.09.06.R,1905.09.06.R,815.0,NaN,NaN


### Indeed there is!
From the previous analysis on 'case_numer' filling pattern, we will use the date formatted as 'YYYY.MM.DD' to fill this recording. But first, let´s check if it will not be duplicated.

In [20]:
df[df['case_number'] == '1905.09.06']

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23


In [21]:
to_update = df[df['case_number'].isnull()].index
to_update

Int64Index([5488], dtype='int64')

In [22]:
df.loc[to_update, 'case_number'] = '1905.09.06'

In [23]:
df[df['case_number']=='1905.09.06']

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
5488,1905.09.06,Reported 06-Sep-1905,1905.0,Provoked,USA,Florida,"Fort Pierce, St Lucie County",Fishing,David Curry,M,...,NaN,"Muncie Evening Press, 8/6/1913",1905.09.06.R-Curry.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1905.09.06.R,1905.09.06.R,815.0,NaN,NaN


### Now it is perfect!!!

* We can see below that the unique and duplicated values are all the registers there is on 'case_number'

In [24]:
df['case_number'].nunique() + \
df[df.duplicated(subset='case_number')]['case_number'].count() - \
len(df)

0

### But, we still have to deal with the duplicated cases and find a logic to exclude or adjust.
### Lets take a better look at these cases:
* First, it is worthwhile to check if there are cases in wich all the columns have the same value:

In [25]:
df.duplicated().sum()

0

In [26]:
duplicated = df[df.duplicated(subset='case_number', keep = False)]

duplicated.nunique().sort_values(ascending=False)

original_order            32
href                      30
href_formula              30
pdf                       30
injury                    27
investigator_or_source    27
location                  26
name                      23
date                      19
year                      17
case_number               16
case_number.1             16
case_number.2             16
area                      16
species                   14
activity                  14
age                       13
time                      11
country                   11
type                       3
fatal_(y/n)                2
sex                        2
unnamed:_22                0
unnamed:_23                0
dtype: int64

### The data above demonstrates that differences are multiple. 
### From a more detailed analysis it was noticed that the same attack might:
* have been reported by differents sources
* have different registers for different people involved

Given the circumstances, it will be considered that a unique register have a unique combination of:

* **case_number**
* __location__
* __name__

So, the the registers that are unique will be mantain and have its case_number adjusted. The duplicated will be deleted

In [27]:
exclude_duplicated = df[df.duplicated(subset=['case_number', 'location', 'name'])].index

df.drop(exclude_duplicated, inplace = True)

In [28]:
# just 2 recordings exlcuded :(
len(df)

6301

In [29]:
# The others rows will be maintained, following the stablished criteria
rename_duplicated = df[df.duplicated(subset=['case_number'])].index

In [30]:
# Adjusting "case_number" for duplicated cases
df.loc[rename_duplicated, 'case_number'] = df.loc[rename_duplicated, 'case_number'].apply(lambda x: x+'D')

# Let the game begin!
### The most relevant registers were selected
Now, the efforts will be in order to **standarize** and **classify** the entries of a same column\
This way wil be possible to use different segmentations and obtain insights from this bunch of words

In [31]:
df.nunique().sort_values(ascending = False)

case_number               6301
original_order            6299
pdf                       6291
href_formula              6290
case_number.2             6286
case_number.1             6285
href                      6285
date                      5433
name                      5230
investigator_or_source    4969
location                  4108
injury                    3736
species                   1549
activity                  1532
area                       825
time                       366
year                       249
country                    212
age                        157
fatal_(y/n)                  8
type                         8
sex                          6
unnamed:_23                  2
unnamed:_22                  1
dtype: int64

### As mentioned before, it looks like most of the columns do not have a filling pattern
Some of them might offer more value information to than others.
### So this work intends to standarize:

* ___activity___
* ___injury___
* ___country___
* ___species___
* ___date___
* ___time___
* ___sex___
* ___area___

### Let´s start by de column  ___'activity'___ 

In [32]:
df['activity'].value_counts()

Surfing                                     971
Swimming                                    869
Fishing                                     430
Spearfishing                                333
Bathing                                     162
                                           ... 
Swimming / floating                           1
Fishing from prawn trawler                    1
Fishing from market fishboat Sea Spray        1
Attempting to net shark in shark channel      1
Lying prone in 2' of water                    1
Name: activity, Length: 1532, dtype: int64

### From the cell above:

* There is a relevant number of standarized classifications. These classification are verbs wich end with the pattern "ing"
* The activities with just one occurence are phrases, so isolte the verb of the phrase may be a good option

In [33]:
def cat(campo):
    campo = str(campo).lower()
    campo = campo.split(" ")
    campo = [re.sub('\W', "", word) for word in campo]
    for word in campo:
        if re.search('ing', word) != None:
            return word
        else:
            return 'unknow'

In [34]:
df['act_cat'] = df['activity'].apply(cat)

In [35]:
df['act_cat'].value_counts().sort_values()

painting           1
restraining        1
lashing            1
crawling           1
scurfing           1
                ... 
spearfishing     406
fishing          612
surfing         1013
swimming        1072
unknow          1718
Name: act_cat, Length: 173, dtype: int64

### It looks good enough!!! 
* For cases in wich the verb does not describe much, there will be the column 'activity' for more details

### Let´s take a look at ___'injury'___

In [36]:
df['injury'].value_counts()

FATAL                                                                   802
Survived                                                                 97
Foot bitten                                                              87
No injury                                                                82
Leg bitten                                                               72
                                                                       ... 
Single laceration on ankle & board bitten                                 1
Thighs bitten, right hand lacerated                                       1
Right arm & right leg bitten                                              1
No injury, minor damage to wetsuit                                        1
No injury; 2 m shark made threat display and impaled itself on spear      1
Name: injury, Length: 3736, dtype: int64

In [37]:
len(df['injury'].value_counts())

3736

In [38]:
df['injury'].isnull().sum()

29

In [39]:
(df['injury'].value_counts()==1).sum()

3348

In [40]:
len(df)

6301

### Lots of unique values that needs to be standarized

#### In order to complete that, let´s consider to be standard: 

* Registers with up to 4 words
* Registers with more than 1 occurences

In [41]:
df[df['injury'].isnull()] = 'unknow'

### Indentifying the keywords:

In [42]:
# Transforming the registers in lists of words:
std_injury = [register.split() for register in df['injury'] if register]

# Bringing all the data to the one unique list and removing non word characters:
std_injury = [re.sub('\W', "", word).lower() for register in std_injury for word in register]

# Making a dict with the number of occurences:
std_injury = { word : std_injury.count(word) for word in set(std_injury) }

# Removing words that do not say much:
dispensable_words = ["", "to", "and", "the", "in", "on", "shark"]
for word in dispensable_words:
    std_injury.pop(word)

### Creating a function to store the most relevant words based on frequency in other registers

In [43]:
def classify(campo):
    campo = str(campo).lower()
    campo = campo.split(" ")
    campo = [re.sub('\W', "", word) for word in campo]
    ranking = { word : std_injury[word] for word in campo if word in std_injury.keys()}
    ranking = sorted(ranking.items(), key = lambda x : x[1], reverse = True)
    ranking = ranking[:4]
    ranking = [double[0] for double in ranking]
    return ranking

In [44]:
df['injury_kw'] = df['injury'].apply(classify)

### Does it make sense?

In [45]:
df[['injury', 'injury_kw' ]].head()

,injury,injury_kw
0,"No injury to occupant, outrigger canoe and pad...","[injury, no, occupant, damaged]"
1,Minor injury to left thigh,"[injury, left, thigh, minor]"
2,Injury to left lower leg from surfboard skeg,"[injury, leg, left, lower]"
3,Minor injury to lower leg,"[injury, leg, minor, lower]"
4,Lacerations to leg & hand shark PROVOKED INCIDENT,"[leg, lacerations, provoked, incident]"


### It seems that the combining words "no" & "injury" might be a good replace for longer statements 

In [46]:
df['injury_kw'].apply(lambda x : ('no' in x) & ('injury' in x)).sum()

804

In [47]:
df['injury'].apply(lambda x : ('no' in x) & ('injury' in x)).sum()

202

In [48]:
# Storing the detailed data for safety 
df['injury_details'] = df['injury'] 

In [49]:
replace_no_injury = df[df['injury_kw'].apply(lambda x : ('no' in x) & ('injury' in x))].index

df.loc[list(replace_no_injury), 'injury'] = 'no injury'

df['injury'].value_counts()

no injury                                           804
FATAL                                               802
Survived                                             97
Foot bitten                                          87
Leg bitten                                           72
                                                   ... 
Ankle and thigh bitten; shark made three strikes      1
Lacerations to right forearm and hand                 1
Puncture wounds on knee                               1
Minor injury to left thigh                            1
Left calf & heel bitten                               1
Name: injury, Length: 3158, dtype: int64

### Oops, forgot the firs step: Normalize what was already there

In [50]:
df['injury'] = df['injury'].apply(str.lower)

In [51]:
df['injury'].value_counts()

fatal                                                                                 804
no injury                                                                             804
survived                                                                               97
foot bitten                                                                            87
leg bitten                                                                             72
                                                                                     ... 
bitten by netted shark provoked incident                                                1
minor injuries to left leg and hand                                                     1
speared shark bit gunwale of boat as it was being hauled onboard provoked incident      1
shark that he speared bit his calf     provoked incident                                1
left forearm bitten, surgically amputated                                               1
Name: inju

## Now it is time for ___"species"___ column

In [52]:
df['species'].value_counts()

White shark                                           162
Shark involvement prior to death was not confirmed    105
Invalid                                               102
Shark involvement not confirmed                        87
Tiger shark                                            73
                                                     ... 
"grey shark"                                            1
a small hammerhead shark                                1
Nurse shark, 0.94 m to 1.2 m [3' to 4']                 1
Tiger shark, 2m                                         1
6' to 8' bull shark                                     1
Name: species, Length: 1547, dtype: int64

In [53]:
no_species = df[df['species'].isnull()].index

len(no_species)

2820

In [54]:
df.loc[list(no_species), 'species'] = 'unknow'

In [55]:
df['species']

0            White shark
1                 unknow
2                 unknow
3              2 m shark
4        Tiger shark, 3m
              ...       
6298              unknow
6299              unknow
6300              unknow
6301              unknow
25722             unknow
Name: species, Length: 6301, dtype: object

### It looks like a similar approach one would fit well

In [56]:
df['species'] =  df['species'].apply(str.lower)

In [57]:
# Transforming the registers in lists of words:
std_spec = [register.split() for register in df['species'] if register]

# Bringing all the data to the one unique list and removing non word characters:
std_spec = [re.sub('\W', "", word).lower() for register in std_spec for word in register]

# Making a dict with the number of occurences:
std_spec = { word : std_spec.count(word) for word in set(std_spec) }

# Removing words that do not say much:
dispensable_words = ["", "to", "and", "the", "in", "on","was", "by"]
for word in dispensable_words:
    try:
        std_injury.pop(word)
    except:
        pass

In [58]:
def classify_spec(campo):
    campo = str(campo).lower()
    campo = campo.split(" ")
    campo = [re.sub("\W", "", word) for word in campo]
    ranking = { word : std_injury[word] for word in campo if word in std_injury.keys()}
    ranking = sorted(ranking.items(), key = lambda x : x[1], reverse = True)
    ranking = ranking[:3]
    ranking = [double[0] for double in ranking]
    return ranking

In [59]:
df['species_kw'] = df['species'].apply(classify)

In [60]:
df[['species', 'species_kw']].head(50)

,species,species_kw
0,white shark,[white]
1,unknow,[unknow]
2,unknow,[unknow]
3,2 m shark,"[2, m]"
4,"tiger shark, 3m","[tiger, 3m]"
5,unknow,[unknow]
6,tiger shark,[tiger]
7,"lemon shark, 3'",[3]
8,"bull shark, 6'","[6, bull]"
9,unknow,[unknow]


In [61]:
df['species_kw'].apply(lambda x: "not" in x).sum()

231

In [62]:
no_shark = df['species'][df['species_kw'].apply(lambda x: ("no" in x) ==True)]
no_shark

125                                 no shark invovlement 
151        no shark invovlement - it ws a publicity stunt
306                                  no shark involvement
307                                  no shark involvement
344                                  no shark involvement
392                                  no shark involvement
438                                  no shark involvement
459                                  no shark involvement
499                                  no shark involvement
558                                  no shark involvement
576                                  no shark involvement
633                                  no shark involvement
690                                  no shark involvement
721                                  no shark involvement
771                                  no shark involvement
778                                  no shark involvement
859                                  no shark involvement
1064          

In [63]:
not_confirmed = df['species'][df['species_kw'].apply(lambda x: ("not" in x) ==True)]
not_confirmed

16                        shark involvement not confirmed
31                        shark involvement not confirmed
44          injury believed caused by an eel, not a shark
115                       shark involvement not confirmed
124        shark involvement prior to death not confirmed
                              ...                        
5486                      shark involvement not confirmed
5493    shark involvement prior to death was not confi...
5577                      shark involvement not confirmed
5578                      shark involvement not confirmed
6147              description of shark does not ring true
Name: species, Length: 231, dtype: object

### Again, some words that offers a reclassification on the original column, so...

In [64]:
df['species_details'] = df['species']

In [65]:
df.loc[list(not_confirmed.index), 'species'] = 'shark involvement not confirmed'

In [66]:
df.loc[list(no_shark.index), 'species'] = 'no shark involvement'

In [67]:
df['species'].value_counts()

unknow                                     2849
shark involvement not confirmed             231
white shark                                 163
invalid                                     102
tiger shark                                  73
                                           ... 
tiger shark, 8 to 10 feet                     1
4.9 m to 5.5 m [16' to 18'] white shark       1
white shark, 3.6 m,  420 kg male              1
bronze whaler shark, 2.4 m [8']               1
"attacked by a number of sharks"              1
Name: species, Length: 1508, dtype: int64

### Defining a function to retrieve numbers from species definition

In [68]:
def sizing (row):
    # get a list of words
    spec = row.split(" ")
    # filter just numbers (decimals and integers) plus possible measures:
    size = [word for word in spec if re.search(r"^\d*[.,]?\d*[-]?[mkglbcm]*['\"]?$",word) != None]
    # calculate the average for cases that have more than one numeric values
    if len(size)>0:
        size = [re.findall('\d+',word) for word in size]
        size = [float(number) for lista in size for number in lista]
        try:
            return round(
                         sum(size)/len(size),
                         1)    
        except:
            pass

In [69]:
df['size'] = df['species'].apply(sizing)

In [70]:
df['size'].value_counts()

3.0      356
4.5      214
4.0      180
5.0      179
2.0      122
        ... 
157.0      1
96.3       1
26.0       1
33.0       1
56.0       1
Name: size, Length: 137, dtype: int64

### Defining function to identify measurement unity

In [71]:
def measure (row):
    spec = row.split(" ")
    # filter just numbers (decimals and integers) plus possible measures:
    size = [word for word in spec if re.search(r"^\d*[.,]?\d*[-]?[mkglbcm]*[feet$]*[foot$]*['\"]?$",word) != None]
    # calculate the average for cases that have more than one numeric values
    irrelevant_words = ['to', 'of', 'be', 'by','too', '']
    if len(size)>0:
        size = [re.findall('[A-Za-z\'"]+',word) for word in size]
        size = [word for lista in size for word in lista if word not in irrelevant_words]
        size = list(set(size))
        return(size) 

In [72]:
df['size_measure'] = df['species'].apply(measure)

In [73]:
df['size_measure'].apply(str).value_counts()

None                4091
['m']               1281
["'"]                586
[]                   204
['lb']                35
['kg']                32
['m', 'lb']           15
['m', 'kg']           13
['cm']                13
['"']                  9
['feet']               5
['foot']               4
['m', 'k']             4
['lb', "'"]            2
['m', "'"]             1
['t', 'm', 'kg']       1
['mm']                 1
['llb']                1
['k', "'"]             1
['left', 'cm']         1
['ft']                 1
Name: size_measure, dtype: int64

### There are cases in wich more than one measurement was registered

* Let´s choose between then. For sake os simplicity, the preference will be given to meters

### STILL NEEDS SOME CLEANING ON DOUBLE MEASUREMENT CASES

### I will leave this for a next time due its significance 

## DATE IS MORE IMPORTANT

In [74]:
df.loc[df['date'].apply(type)==float]

,case_number,date,year,type,country,area,location,activity,name,sex,...,original_order,unnamed:_22,unnamed:_23,act_cat,injury_kw,injury_details,species_kw,species_details,size,size_measure


In [75]:
df.drop(index=25722, inplace=True)

In [76]:
df['year_aux'] = df['date'].apply(lambda x : re.findall('[0-9][0-9][0-9][0-9]{0,1}', x))

In [77]:
def str_to_int (lista) :
    lista = [int(item) for item in lista]
    if len(lista) == 0:
        lista = [0]
        return 
    else:
        return(lista)

In [78]:
df['year_avg'] = df['year_aux'].apply(str_to_int)

In [79]:
df['year_avg'].apply(type)

0       <class 'list'>
1       <class 'list'>
2       <class 'list'>
3       <class 'list'>
4       <class 'list'>
             ...      
6297    <class 'list'>
6298    <class 'list'>
6299    <class 'list'>
6300    <class 'list'>
6301    <class 'list'>
Name: year_avg, Length: 6300, dtype: object

In [80]:
indexes = df[df['year_avg'].isnull()].index

In [81]:
df.loc[list(indexes), 'year_avg'] = 0

In [82]:
def avg_year (row):
    try:
        soma = sum(row)
        itens =  len(row)
        if (soma > 0) & (itens > 0):
            year = int((soma/itens))
            return  year
    except:
        return 'unknow'

In [83]:
df['year_avg'] =df['year_avg'].apply(avg_year)

In [84]:
df

,case_number,date,year,type,country,area,location,activity,name,sex,...,unnamed:_23,act_cat,injury_kw,injury_details,species_kw,species_details,size,size_measure,year_aux,year_avg
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,NaN,paddling,"[injury, no, occupant, damaged]","No injury to occupant, outrigger canoe and pad...",[white],white shark,NaN,None,[2018],2018
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,standing,"[injury, left, thigh, minor]",Minor injury to left thigh,[unknow],unknow,NaN,None,[2018],2018
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,surfing,"[injury, leg, left, lower]",Injury to left lower leg from surfboard skeg,[unknow],unknow,NaN,None,[2018],2018
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,NaN,surfing,"[injury, leg, minor, lower]",Minor injury to lower leg,"[2, m]",2 m shark,2.0,[m],[2018],2018
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,NaN,unknow,"[leg, lacerations, provoked, incident]",Lacerations to leg & hand shark PROVOKED INCIDENT,"[tiger, 3m]","tiger shark, 3m",3.0,[m],[2018],2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,NaN,diving,[fatal],FATAL,[unknow],unknow,NaN,None,[1903],1903
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,NaN,unknow,[fatal],FATAL,[unknow],unknow,NaN,None,[1903],1903
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,NaN,swimming,[fatal],FATAL,[unknow],unknow,NaN,None,"[1900, 1905]",1902
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,...,NaN,unknow,[fatal],FATAL,[unknow],unknow,NaN,None,"[1883, 1889]",1886


In [85]:
def before_jc (row):
    marking = re.search('b.c.', row, re.I)
    if marking != None:
        return 'B.C.'
    else:
        return 'A.D.'

In [86]:
df['ad/bc'] = str()

In [87]:
df['ad/bc'] = df['date'].apply(before_jc)

In [88]:
df[df['ad/bc']=='B.C.']

,case_number,date,year,type,country,area,location,activity,name,sex,...,act_cat,injury_kw,injury_details,species_kw,species_details,size,size_measure,year_aux,year_avg,ad/bc
6177,0000.0214,Ca. 214 B.C.,0.0,Unprovoked,NaN,Ionian Sea,NaN,Ascending from a dive,"Tharsys, a sponge diver",M,...,ascending,"[fatal, sharks, bit, him]","FATAL, shark/s bit him in two",[unknow],unknow,NaN,None,[214],214,B.C.
6178,0000.0336,Ca. 336.B.C..,0.0,Unprovoked,GREECE,Piraeus,In the haven of Cantharus,Washing his pig in preparation for a religious...,A candidate for initiation,M,...,washing,"[fatal, of, bit, lower]","FATAL, shark ""bit off all lower parts of him u...",[unknow],unknow,NaN,None,[336],336,B.C.
6179,0000.0493,493 B.C.,0.0,Sea Disaster,GREECE,Off Thessaly,NaN,Shipwrecked Persian Fleet,males,M,...,unknow,"[of, sharks, water, men]",Herodotus tells of sharks attacking men in the...,[unknow],unknow,NaN,None,[493],493,B.C.
6180,0000.0725,Ca. 725 B.C.,0.0,Sea Disaster,ITALY,Tyrrhenian Sea,Krater found during excavations at Lacco Ameno...,Shipwreck,males,M,...,unknow,"[by, sharks, a, attacked]",Depicts shipwrecked sailors attacked by a sha...,[unknow],unknow,NaN,None,[725],725,B.C.


In [89]:
years = list(([year for year in df['year_avg'] if year != 'unknow']))

In [90]:
max(years)

2176

In [102]:
df[df['year_avg']==2176]

,case_number,date,year,type,country,area,location,activity,name,sex,...,act_cat,injury_kw,injury_details,species_kw,species_details,size,size_measure,year_aux,year_avg,ad/bc
4078,1959.08.02,21764,1959.0,Invalid,ITALY,Tuscany,"Cala del Corvo, Isola del Giglio",Scuba diving,Karl Pollerer & Eric Eisesenid,M,...,unknow,"[involvement, probable, unconfirmed, drowing]",Probable drowing. Shark involvement unconfirmed,"[not, death, involvement, prior]",shark involvement prior to death was not confi...,NaN,None,[2176],2176,A.D.


In [ ]:

years = df[df['ad/bc'] == 'B.C.'].index

In [ ]:
years = df.loc[years,'year_avg']

In [ ]:
lista = list(df[df['year_avg'].apply(type) == int].index)

In [ ]:
df1 = df.loc[lista]

In [ ]:
(df1['year_avg']>2020).value_counts()

In [1]:
df.loc[4078,'year_avg'] = 1959

NameError: name 'df' is not defined

In [ ]:
path = os.path.expanduser('~/Desktop/SharAttack/')

In [ ]:
df.to_excel(f'{path}db_attack.xlsx')